In [1]:
country = 'CH'
country_name = 'switzerland'

In [2]:
import json

In [3]:
with open('../europe.json') as f:
    model = json.load(f)

In [4]:
{id: len(val) for id, val in model.items() if isinstance(val, dict)}

{'bus': 4097,
 'dcline': 0,
 'gen': 1083,
 'branch': 8375,
 'storage': 0,
 'switch': 0,
 'shunt': 0,
 'load': 4097}

Delete all foreign buses:

In [5]:
for id in [id for id, bus in model['bus'].items() if bus['country'] != country]:
    del model['bus'][id]

Delete all foreign lines (including lines across the border):

In [6]:
for id in [id for id, line in model['branch'].items()
           if str(line['f_bus']) not in model['bus'] or str(line['t_bus']) not in model['bus']]:
    del model['branch'][id]

Check whether all nodes are still connected to the rest of the model:

In [7]:
bus_degree = {id: 0 for id in model['bus'].keys()}
for line in model['branch'].values():
    bus_degree[str(line['f_bus'])] += 1
    bus_degree[str(line['t_bus'])] += 1

In [8]:
disconnected_buses = {id for id, degree in bus_degree.items() if degree == 0}
disconnected_buses

{'5628'}

In [9]:
for id in disconnected_buses:
    del model['bus'][id]
    del bus_degree[id]

In [10]:
disconnected_pairs = {(id, (str(line['f_bus']), str(line['t_bus'])))
                      for id, line in model['branch'].items()
                      if bus_degree[str(line['f_bus'])] == 1 and bus_degree[str(line['t_bus'])] == 1}
disconnected_pairs

{('7974', ('5115', '5116'))}

In [11]:
for line_id, (bus_id_1, bus_id_2) in disconnected_pairs:
    del model['branch'][line_id]
    del model['bus'][bus_id_1]
    del model['bus'][bus_id_2]
    del bus_degree[bus_id_1]
    del bus_degree[bus_id_2]

Delete all foreign loads:

In [12]:
for id in [id for id, load in model['load'].items() if str(load['load_bus']) not in model['bus']]:
    del model['load'][id]

Delete all foreign gens:

In [13]:
for id in [id for id, gen in model['gen'].items() if str(gen['gen_bus']) not in model['bus']]:
    del model['gen'][id]

Remaining model:

In [14]:
{id: len(val) for id, val in model.items() if isinstance(val, dict)}

{'bus': 160,
 'dcline': 0,
 'gen': 36,
 'branch': 330,
 'storage': 0,
 'switch': 0,
 'shunt': 0,
 'load': 160}

Export:

In [15]:
model_json = json.dumps(model)
with open('../%s.json' % country_name, 'w') as f:
    f.write(model_json)